In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import json
import ast
import os
import matplotlib
matplotlib.rcParams.update({'font.size': 22})

In [ ]:
val_losses_big_dict = {}
test_losses_big_dict = {}
train_losses_big_dict = {}

filenames = ["../Data/"+x for x in os.listdir("../Data/")]
for filename in filenames:
    if "cali" in filename:
        if "test" in filename:
            # Load the data from your JSON file (replace 'data.json' with your file path)
            with open(filename, 'r') as file:
                test_losses_dict = json.load(file)
            test_losses_big_dict[filename] = test_losses_dict
        if "val" in filename:
            # Load the data from your JSON file (replace 'data.json' with your file path)
            with open(filename, 'r') as file:
                val_losses_dict = json.load(file)
            val_losses_big_dict[filename] = val_losses_dict

        if "train" in filename:
            # Load the data from your JSON file (replace 'data.json' with your file path)
            with open(filename, 'r') as file:
                train_losses_dict = json.load(file)
            train_losses_big_dict[filename] = train_losses_dict

In [ ]:
for filename, test_loss_dict in test_losses_big_dict.items():
    test_loss_list = []
    for trial, test_loss in test_loss_dict.items():
        if not np.isnan(test_loss[0]):
            test_loss_list.append(test_loss[0])
    if len(test_loss_list) < 5:
        print("Some NaN losses: n = {}".format(len(test_loss_list)))
    print(filename, "Test MSE: {} +/- {}".format(round(np.mean(test_loss_list), 4), round(np.std(test_loss_list), 4)))
    print("")

In [ ]:
# Train Loss Plotting

factor = 1

colors = {
    "Adam": "b",
    "ExpTest": "g",
    "SGD w/ Mom.": "r",
    "SGD": "c",
    "RMSprop": "m",
    "Adadelta": "k",
}

plt.figure(figsize=(14, 8))
plt.tight_layout()
plt.ylim(bottom=0.1, top=0.5)

for filename, train_loss_dict in train_losses_big_dict.items():
    if ("algo") in filename:
        optimizer_name = "ExpTest"
    elif ("delta") in filename:
        optimizer_name = "Adadelta"
    elif ("Mom") in filename:
        optimizer_name = "SGD w/ Mom."
    elif ("Adam") in filename:
        optimizer_name = "Adam"
    elif ("RMS") in filename:
        optimizer_name = "RMSprop"
    else:
        optimizer_name = "SGD"

    if (f"_{factor}x" in filename) or ("algo" in filename) or ("delta" in filename):
        train_losses_list = []
        for trial, train_losses in train_loss_dict.items():
            train_losses_list.append(train_losses[0])
        losses_array = np.array(train_losses_list)
        mean_loss = np.mean(losses_array, axis=0)
        sem_loss = np.std(losses_array, axis=0) / np.sqrt(losses_array.shape[0])
        iters = np.arange(len(mean_loss))
        if np.any(mean_loss < 0.5):
            plt.plot(iters, mean_loss, label=optimizer_name, color=colors[optimizer_name])
            plt.fill_between(iters, mean_loss - sem_loss, mean_loss + sem_loss, alpha=0.4, color=colors[optimizer_name])

plt.xlabel('Epochs')
plt.ylabel('Training Loss')
leg = plt.legend(ncol=3, loc='lower center')
for legobj in leg.legend_handles:
    legobj.set_linewidth(4)
plt.grid()
plt.savefig(f"Cali_{factor}x.png", dpi=600)
plt.show()

In [ ]:
# Validation Loss Plotting

factor = 1

colors = {
    "Adam": "b",
    "ExpTest": "g",
    "SGD w/ Mom.": "r",
    "SGD": "c",
    "RMSprop": "m",
    "Adadelta": "k",
}

plt.figure(figsize=(14, 8))
plt.tight_layout()
plt.ylim(bottom=0.1, top=0.5)

for filename, val_loss_dict in val_losses_big_dict.items():
    if ("algo") in filename:
        optimizer_name = "ExpTest"
    elif ("delta") in filename:
        optimizer_name = "Adadelta"
    elif ("Mom") in filename:
        optimizer_name = "SGD w/ Mom."
    elif ("Adam") in filename:
        optimizer_name = "Adam"
    elif ("RMS") in filename:
        optimizer_name = "RMSprop"
    else:
        optimizer_name = "SGD"

    if (f"_{factor}x" in filename) or ("algo" in filename) or ("delta" in filename):
        val_losses_list = []
        for trial, val_losses in val_loss_dict.items():
            val_losses_list.append(val_losses[0])
        losses_array = np.array(val_losses_list)
        mean_loss = np.mean(losses_array, axis=0)
        sem_loss = np.std(losses_array, axis=0) / np.sqrt(losses_array.shape[0])
        iters = np.arange(len(mean_loss))
        if np.any(mean_loss < 0.5):
            plt.plot(iters, mean_loss, label=optimizer_name, color=colors[optimizer_name])
            plt.fill_between(iters, mean_loss - sem_loss, mean_loss + sem_loss, alpha=0.4, color=colors[optimizer_name])

plt.xlabel('Epochs')
plt.ylabel('Validation Loss')
leg = plt.legend(ncol=3, loc='lower center')
for legobj in leg.legend_handles:
    legobj.set_linewidth(4)
plt.grid()
plt.savefig(f"Cali_Val_{factor}x.png", dpi=600)
plt.show()